# Test 7: Analyze Truth Jet and MET with Scheme 1
Test my analysis function with scheme 1 of truth jet and MET by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class
Skip:  
2-1. Check the number of events for each branch  
2-2. Define mass quantities

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels
Skip

## 4. Jet Clustering

### 4-1. Select stable final state particles without/with filtering out dark sector

In [3]:
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [4]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v3 = myAnal_v3.jetClustering_v1(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering_v1(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done new version


## 5. Analyze the Jet and MET in the Truth Level

### 5-1. Preselection version 1

In [5]:
presel_bef, presel_pt, presel_pt_eta, presel_idx = myAnal_v3.preselection_v1(PseudoJet_filterDM_v3, pT_min=20, eta_max=2.5)

19373 events before preselection
19373 events after pT preselection
19373 events after pT & eta preselections
--------------------------------------------------------------------------------
0 events without PseudoJet before preselection
357 events without PseudoJet after pT preselection
519 events without PseudoJet after pT & eta preselections


### 5-2. MET

In [6]:
arr_MET, df_MET = myAnal_v3.MET_visParticles_v1(SFSP_filterDM_v3)

19373 events in MET data.


In [7]:
arr_MET[:5]

array([(180.36140062,  2.80660527, -170.33591449,  59.29511843),
       (158.13608629, -0.31722873,  150.24565215, -49.32814406),
       (201.62627625, -0.37612596,  187.53151556, -74.06136607),
       (105.30642437,  2.74217592,  -97.01754218,  40.95167303),
       (190.59392338,  0.52239787,  165.173503  ,  95.09867262)],
      dtype=[('MET', '<f8'), ('phi', '<f8'), ('METx', '<f8'), ('METy', '<f8')])

In [8]:
df_MET.head()

,MET,phi,METx,METy
0,180.361401,2.806605,-170.335914,59.295118
1,158.136086,-0.317229,150.245652,-49.328144
2,201.626276,-0.376126,187.531516,-74.061366
3,105.306424,2.742176,-97.017542,40.951673
4,190.593923,0.522398,165.173503,95.098673


### 5-3. Analyze truth jet and MET with scheme 1 and version 1
The 4-vector of top 3 jets and $\vec{E}_T^\mathrm{miss}$ vector are given by
\begin{align}
    j_1^\mu &= (E_1,\ p_{x,1},\ p_{y,1},\ p_{z,1}) \\
    j_2^\mu &= (E_2,\ p_{x,2},\ p_{y,2},\ p_{z,2}) \\
    j_3^\mu &= (E_3,\ p_{x,3},\ p_{y,3},\ p_{z,3}) \\
    \vec{E}_T^\mathrm{miss} &= (\mathrm{MET},\ \mathrm{MET}_x,\ \mathrm{MET}_y,\ \mathrm{MET}_z = 0) \to\text{This is not 4-vector.}
\end{align}
Calculate 4-vectors of dijet and trijet:
\begin{align}
    jj^\mu &= \big(E(jj),\ p_x(jj),\ p_y(jj),\ p_z(jj)\big) = (E_1+E_2,\ p_{x,1}+p_{x,2},\ p_{y,1}+p_{y,2},\ p_{z,1}+p_{z,2}) \\
    jjj^\mu &= \big(E(jjj),\ p_x(jjj),\ p_y(jjj),\ p_z(jjj)\big) =
    (E_1+E_2+E_3,\ p_{x,1}+p_{x,2}+p_{x,3},\ p_{y,1}+p_{y,2}+p_{y,3},\ p_{z,1}+p_{z,2}+p_{z,3})
\end{align}
The transverse mass $M_T(\mathrm{jets}, \mathrm{MET})$ is given by
\begin{align}
    M_T^2(jj,\vec{E}_T^\mathrm{miss}) = [E_T(jj) + \mathrm{MET}]^2 - [p_x(jj) + \mathrm{MET}_x]^2 - [p_y(jj) + \mathrm{MET}_y]^2
\end{align}
where
\begin{align}
    E_T^2(jj) = m^2(jj) + p_T^2(jj)
\end{align}

In [9]:
df_jj_MET, df_jjj_MET = myAnal_v3.analyze_truthJet_MET_scheme1_v1(presel_pt_eta, arr_MET)

In [10]:
df_jj_MET

,MT_jj_MET,Dphi_jj_MET,Dphi_j1_MET,Dphi_j2_MET,min_Dphi_j_MET,selected
0,362.830698,3.139757,3.052305,2.724209,2.724209,0.0
1,468.548803,2.519317,1.770399,2.603022,1.769629,1.0
2,509.241834,3.021533,2.887976,2.955023,0.096783,4.0
3,745.457867,2.406662,2.298874,0.989178,0.273139,5.0
4,273.625077,2.908264,3.086091,2.719188,0.724000,7.0
...,...,...,...,...,...,...
16747,1110.778285,3.117308,3.139996,2.621158,2.621158,19367.0
16748,873.863409,3.074515,2.647485,1.704960,0.026529,19369.0
16749,1165.460014,1.471860,2.494809,0.300742,0.300742,19370.0
16750,145.967120,2.105484,2.656224,1.486128,1.076084,19371.0


In [11]:
df_jjj_MET.head(3)

,MT_jjj_MET,Dphi_jjj_MET,Dphi_j1_MET,Dphi_j2_MET,Dphi_j3_MET,min_Dphi_j_MET,selected
0,628.598105,3.109111,1.770399,2.603022,1.769629,1.769629,1.0
1,589.947743,3.015251,2.887976,2.955023,0.096783,0.096783,4.0
2,993.492687,2.870788,2.298874,0.989178,1.772532,0.273139,5.0


## 6. Test

In [12]:
x = np.array([1, 2, 3, 4, 5])
print("x = ", x)
y = abs(x - 10)
print("y = ", y)
print("y<8 = ", y[y < 8])
y[y < 8] = 10 + y[y < 8]
y

x =  [1 2 3 4 5]
y =  [9 8 7 6 5]
y<8 =  [7 6 5]


array([ 9,  8, 17, 16, 15])

In [13]:
y[:4]

array([ 9,  8, 17, 16])

In [14]:
np.min(y[:4])

8

In [15]:
x = np.array([2, 1])
x

array([2, 1])

In [16]:
x[:4]

array([2, 1])

In [17]:
np.min(x[:4])

1

In [18]:
x = np.array([1, 2, 3, 4, 5, 6, 7])
x

array([1, 2, 3, 4, 5, 6, 7])

In [19]:
x[:3]

array([1, 2, 3])

In [20]:
np.sum(x[:3])

6